# Bruk av geopandas med data fra NVDB og SSB

Demo av hvordan geopandas kan brukes med data fra [Nasjonal vegdatabank](https://www.vegvesen.no/fag/teknologi/Nasjonal+vegdatabank) (NVDB) og [Statistisk Sentralbyrå](http://www.ssb.no/natur-og-miljo/geodata) (SSB). 

Vi ønsker å finne hvilke turistveger som går gjennom hvilke sentrumsområder - og hvilke trafikkulykker som er registrert der. 

Gangen er som følger: 
1. Last ned data over sentrumsområder fra [SSB](http://www.ssb.no/natur-og-miljo/geodata
1. Last ned turistveger fra [NVDB api](https://www.vegvesen.no/nvdb/apidokumentasjon/)
1. Finn snittet av turistveger og sentrumsområder (hvilke turistveger som går gjennom hvilke sentrumsområder)
1. Finn trafikkulykkene på turistvegene som går gjennom sentrumsområder. 
    1. Finn BoundingBox for hvert av sentrumsområdene 
    1. Hent trafikkulykker innenfor BBox. M
    1. Finn trafikkulykkene som ligger på turistvegens senterlinje
        1. Alternativt finn trafikkulykkene innenfor radius fra turistvegen. 
1. Lag fine plott og sånn


# Last ned data fra SSB

Dette steget er overflødig om du vil laste ned manuelt. *NB! Det vil ikke overraske meg om lenkeråte gjør at du uansett må inn på SSB og finne nedlastingslenker* 

Gå inn på http://www.ssb.no/natur-og-miljo/geodata


In [ ]:
import requests
import shutil 

url = 

# Se https://stackoverflow.com/a/39217788 


